In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.7 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [6]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [7]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [24]:
import pandas as pd
import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd drive/MyDrive/n4xx/data/data

/content/drive/MyDrive/n4xx/data/data


In [54]:
from glob import glob
import json
import re

#방송 대본 질의 형식으로 데이터프레임에 저장하기
Q=[]
A=[]
for filename in glob('*.json'):
  with open(filename, "r") as json_file:
    passage = json.load(json_file)["Meta"]["passage"]
    sentenses=passage.split("\n")
    sentenses=[s for s in sentenses if "]" in s and "해설]" not in s]
    trim_sentenses=[]
    for sentense in sentenses:
      trim_sentense=re.sub(r'\([^)]*\)',"",sentense.split("]")[1]).strip()
      #공백이나 .같은 의미없는 데이터 제외
      if len(trim_sentense)>1:
        trim_sentenses.append(trim_sentense)
    Q+=trim_sentenses[:-1]
    A+=trim_sentenses[1:]

df=pd.DataFrame({'Q':Q,'A':A})
df

,Q,A
0,"걱정 말어, 시장이 이번 선거에 적극 나섰디야. 각 동마다 동사무소장헌티 여당 찍어...",이제 결관 하늘에 맡길 겁니다.
1,이제 결관 하늘에 맡길 겁니다.,잘 될 거여. 걱정 말어.
2,"형님. 왜 그래. 아유, 철없는 자식이 한 말에 왜 마음을 써!",...으흐흑.
3,...으흐흑.,형님...
4,형님...,내가 지를 얼마나 이뻐했는데... 내 아들이나 마찬가지라고 생각하고 애지중지 사랑했...
...,...,...
78117,"할머니, 이상해. 몸하고 마음이 따로 따로야. 마음은 수희한테 져주고도 싶은데, 몸...",뭐든 넉넉하게 넣어주세요. 사람들이 여럿이잖아요.
78118,뭐든 넉넉하게 넣어주세요. 사람들이 여럿이잖아요.,참 잡지나 신문도 널 수 있어요?
78119,참 잡지나 신문도 널 수 있어요?,"여기선 며칠 안 계신다며요, 그런 건 나중에,"
78120,"여기선 며칠 안 계신다며요, 그런 건 나중에,",할머니가 말씀을 통안 하세요. 힘들겠어요.


In [24]:
len(df)

78122

In [22]:
batch_size = 32

In [38]:
def get_chat_data():
  for question, answer in zip(df["Q"].to_list(), df["A"].to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer, max_length=40) 
    yield bos_token + sent + eos_token

In [39]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [40]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [41]:
for batch in dataset:
    print(batch)
    break

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tf.Tensor(
[[    1     2 30806 ...     1     3     3]
 [    1     2  9873 ...     3     3     3]
 [    1     2   739 ... 25462     1     3]
 ...
 [    1     2   739 ...     3     3     3]
 [    1     2 46651 ...     3     3     3]
 [    1     2 23090 ...   739  6910     1]], shape=(32, 42), dtype=int32)


In [42]:
tokenizer.decode(batch[0])

'</s><usr> 임시로 가흔씨 옷으로 베개 만들어줬어요. 그랬더니 잘 자더라구요.<sys> 네.. 그래도 엄마가 챙겨준 베개가 더 편할 거예요.</s><pad><pad>'

In [30]:
print(batch[0])

tf.Tensor(
[    1     2 30806  9028  8797  7957 11019  9021  9538  6841  9659  8245
  8006 25856 37202 18952  9443  9042  7208  7372  6919 25856     4  9873
  9705 31416 10287 16686 42226  6874  8238  9538 12419  9267  9266  8705
  9122  8046 25856     1     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3], shape=(83,), dtype=int32)


In [43]:
print(tokenizer.encode('</s><usr> 임시로 가흔씨 옷으로 베개 만들어줬어요. 그랬더니 잘 자더라구요.<sys> 네.. 그래도 엄마가 챙겨준 베개가 더 편할 거예요.</s><pad><pad>'))

[1, 2, 30806, 9028, 8797, 7957, 11019, 9021, 9538, 6841, 9659, 8245, 8006, 25856, 37202, 18952, 9443, 9042, 7208, 7372, 6919, 25856, 4, 9873, 9705, 31416, 10287, 16686, 42226, 6874, 8238, 9538, 12419, 9267, 9266, 8705, 9122, 8046, 25856, 1, 3, 3]


In [44]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [45]:
steps = len(df) // batch_size + 1
print(steps)

2442


In [48]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/2442 [00:00<?, ?it/s]

[Epoch:    1] cost = 1.2023778


  0%|          | 0/2442 [00:00<?, ?it/s]

[Epoch:    2] cost = 0.998561919


  0%|          | 0/2442 [00:00<?, ?it/s]

[Epoch:    3] cost = 0.835529327


In [133]:
model.save_pretrained("myGPT_mode.h5")

In [8]:
loaded_model=TFGPT2LMHeadModel.from_pretrained("myGPT_mode.h5")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at myGPT_mode.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [11]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [139]:
def preprocess_sentence2(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, max_length=40)
  input_ids = tf.convert_to_tensor([input_ids])
  output = loaded_model.generate(input_ids, max_length=40, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  print(len(output))
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [142]:
preprocess_sentence2('고민이 있어요.')

1


'무슨 소리야?'

In [108]:
preprocess_sentence2('게임하고싶은데 할래?')

1


'그럼요.'

In [109]:
preprocess_sentence2('나 너 좋아하는 것 같아')

1


'왜에요!'

In [110]:
preprocess_sentence2('그만 헤어지자')

1


'그래. 오늘 파티가 제일 중요해. 신브랜드 담당자인 내가 보고 싶어서 와도 못 갈 거 같아.'

In [112]:
preprocess_sentence2('너도 똑같은 놈이야')

1


'뭐가 왜 쌍으로 무슨 주둥이 주둥이 주둥이!'

In [148]:
preprocess_sentence2('너를 너무 보고 싶었어.')

1


'난 안 믿었으니까 믿으라고.'